In [4]:
from bert_embedding import BertEmbedding
import pandas as pd
import numpy as np
import mxnet as mx

In [2]:
fp = open('./Amazon_Instant_Video/OnlyText.txt','r')
lines = fp.readlines()

In [3]:
text_data = []
for line in lines:
    text_data.append(line.strip('\n'))

In [14]:
print(len(text_data))

50000


In [7]:
ctx = mx.gpu(0)
bert = BertEmbedding(ctx=ctx)

/home/subbareddy/anaconda3/lib/python3.6/site-packages/gluonnlp/model/block.py:113: UserWarning: `erf` operator support is not found. Please consider upgrading to mxnet >= 1.4
  warnings.warn('`erf` operator support is not found. '


In [8]:
result = bert.embedding(text_data)

In [12]:
len(result[0][0])

768

In [13]:
x_features = []
for i in np.arange(len(result)):
    x_features.append(result[i][0])

In [14]:
x_features = np.array(x_features)
x_features.shape

(50000, 768)

In [15]:
labels = pd.read_csv('./Amazon_Instant_Video/polarity.txt',header=None)

In [16]:
labels.shape

(50000, 1)

In [32]:
y = []
for i in labels[0]:
    if i=='positive':
        y.append(1)
    else:
        y.append(0)

In [33]:
y = np.array(y)

In [37]:
from sklearn.model_selection import train_test_split,cross_val_score
x_train,x_test,y_train,y_test = train_test_split(x_features,y,test_size=0.2,random_state=47)

In [22]:
import lightgbm as lgb

In [35]:
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(x_train,y_train)
y_pred = lgb_model.predict(x_test)

/home/subbareddy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.71      0.76      0.74      4965
          1       0.75      0.69      0.72      5035

avg / total       0.73      0.73      0.73     10000

[[3792 1173]
 [1543 3492]]


In [38]:
print(cross_val_score(lgb_model,x_features,y,cv=5))

/home/subbareddy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/subbareddy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/subbareddy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/subbareddy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an 

[0.7277 0.7168 0.709  0.6997 0.7357]


/home/subbareddy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
